In [1]:
using VCFTool

┌ Info: Precompiling VCFTool [8f8d80d9-e796-45a3-8ef2-c0461a3110a4]
└ @ Base loading.jl:1278


## Read VCF

In [ ]:
input_dir = "../input/"

In [ ]:
vcf_738 = read_vcf(joinpath(input_dir, "738_variants.vcf.gz"))

vcf_example = read_vcf(joinpath(input_dir, "example.vcf.gz"))

vcf_to_use = vcf_738

## Read BED

In [ ]:
using DataFrames
using CSV: File

bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

bed_io = open("../input/cardiotoxicity.bed")

bed = DataFrame(File(io; delim='\t', header=false))

In [ ]:
print_and_run_cmd(`tabix -R `)

## Check Variants

https://cardiooncologyjournal.biomedcentral.com/articles/10.1186/s40959-020-00060-0/tables/3

In [ ]:
# rs1799983

using Query, DataFrames

x = @from i in vcf_to_use begin
    @where i.CHROM=="chr7"
    @where i.POS==150999023
    @select {i.CHROM, i.POS, i.REF, i.ALT, i.FORMAT, i.Germ}
    @collect DataFrame
end

println(x)